In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle (4).json to kaggle (4).json


{'kaggle (4).json': b'{"username":"dipakkhandagale","key":"163005f489516f9231ff065b93374c29"}'}

In [3]:
import os
os.environ['KAGGLE_USERNAME'] = 'dipakkhandagale'
os.environ['KAGGLE_KEY'] = '163005f489516f9231ff065b93374c29'

In [4]:
import kaggle
kaggle.api.authenticate()

In [5]:
!kaggle competitions download -c deepfake-detection-challenge

 98% 4.06G/4.13G [01:05<00:04, 16.7MB/s]
100% 4.13G/4.13G [01:05<00:00, 67.7MB/s]


In [6]:
!unzip deepfake-detection-challenge.zip -d /content/data

Archive:  deepfake-detection-challenge.zip
  inflating: /content/data/sample_submission.csv  
  inflating: /content/data/test_videos/aassnaulhq.mp4  
  inflating: /content/data/test_videos/aayfryxljh.mp4  
  inflating: /content/data/test_videos/acazlolrpz.mp4  
  inflating: /content/data/test_videos/adohdulfwb.mp4  
  inflating: /content/data/test_videos/ahjnxtiamx.mp4  
  inflating: /content/data/test_videos/ajiyrjfyzp.mp4  
  inflating: /content/data/test_videos/aktnlyqpah.mp4  
  inflating: /content/data/test_videos/alrtntfxtd.mp4  
  inflating: /content/data/test_videos/aomqqjipcp.mp4  
  inflating: /content/data/test_videos/apedduehoy.mp4  
  inflating: /content/data/test_videos/apvzjkvnwn.mp4  
  inflating: /content/data/test_videos/aqrsylrzgi.mp4  
  inflating: /content/data/test_videos/axfhbpkdlc.mp4  
  inflating: /content/data/test_videos/ayipraspbn.mp4  
  inflating: /content/data/test_videos/bcbqxhziqz.mp4  
  inflating: /content/data/test_videos/bcvheslzrq.mp4  
  inflatin

In [7]:
!pip install facenet-pytorch efficientnet_pytorch
!pip install Pillow==10.2.0 torchvision==0.17.1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.2 MB/s eta 0:00:00
   ━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.2/755.5 MB 3.4 MB/s eta 0:02:44
ERROR: Operation cancelled by user
^C


In [1]:
import os
orig = '/content/data/train sample videos'
tgt  = '/content/data/train_sample_videos'
if os.path.isdir(orig) and not os.path.isdir(tgt):
    os.rename(orig, tgt)


In [2]:
!pip install torch

In [3]:
import os
import json
from pathlib import Path
import cv2
from PIL import Image
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision import models
from facenet_pytorch import MTCNN

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")


Using device: cuda


In [4]:
VIDEO_DIR = Path('/content/data/train_sample_videos')
if not VIDEO_DIR.exists(): raise FileNotFoundError(f"Video dir not found: {VIDEO_DIR}")

possible_meta = [VIDEO_DIR / 'metadata.json', Path('/content/data/metadata.json')]
META_PATH = next((p for p in possible_meta if p.exists()), None)
if META_PATH is None: raise FileNotFoundError("metadata.json missing.")
print(f"Metadata: {META_PATH}")

FRAMES_DIR = Path('/content/frames')
NUM_FRAMES  = 12  # changed from 16 → 12 to reduce memory load
FRAMES_DIR.mkdir(exist_ok=True)


Metadata: /content/data/train_sample_videos/metadata.json


In [5]:
mtcnn  = MTCNN(image_size=224, margin=20, keep_all=False, device=DEVICE)
to_pil = T.ToPILImage()

for vid in tqdm(sorted(VIDEO_DIR.glob('*.mp4')), desc='Videos'):
    od = FRAMES_DIR / vid.stem
    od.mkdir(exist_ok=True)
    cap   = cv2.VideoCapture(str(vid))
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) or 1
    step  = max(total // NUM_FRAMES, 1)
    saved = 0
    for i in range(NUM_FRAMES):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * step)
        ret, frm = cap.read()
        if not ret: break
        img = Image.fromarray(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))
        face = mtcnn(img)
        if face is None:
            w, h = img.size
            crop = img.crop(((w - h) // 2, 0, (w + h) // 2, h))
            face = T.Resize((224,224))(T.ToTensor()(crop))
        pil = to_pil(face)
        pil.save(od / f'frame_{i:04d}.jpg')
        saved += 1
    cap.release()
    if saved < NUM_FRAMES:
        print(f"⚠ {vid.stem}: {saved}/{NUM_FRAMES}")


Videos:   0%|          | 0/400 [00:00<?, ?it/s]

In [6]:
with open(META_PATH) as f:
    raw_meta = json.load(f)

metadata = {Path(k).stem: raw_meta[k] for k in raw_meta}
names    = list(metadata.keys())

split     = int(0.8 * len(names))
train_keys = names[:split]
val_keys   = names[split:]
train_meta = {k: metadata[k] for k in train_keys}
val_meta   = {k: metadata[k] for k in val_keys}

class FrameDataset(Dataset):
    def __init__(self, root, meta, n_frames=NUM_FRAMES):
        self.root    = Path(root)
        self.meta    = meta
        self.names   = list(meta.keys())
        self.n_frames= n_frames
        self.tf = T.Compose([
            T.RandomHorizontalFlip(p=0.5),
            T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
            T.RandomRotation(degrees=5),
            T.ToTensor(),
            T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
        ])
    def __len__(self):
        return len(self.names)
    def __getitem__(self, idx):
        nm    = self.names[idx]
        label = 1.0 if self.meta[nm]['label'] == 'FAKE' else 0.0
        folder= self.root / nm
        frames_list = []
        for i in range(self.n_frames):
            fp = folder / f'frame_{i:04d}.jpg'
            if not fp.exists():
                raise FileNotFoundError(f"Missing frame: {fp}")
            img = Image.open(fp).convert('RGB')
            frames_list.append(self.tf(img))
        video_tensor = torch.stack(frames_list, dim=0)
        return video_tensor, torch.tensor(label, dtype=torch.float32)

train_ds    = FrameDataset(FRAMES_DIR, train_meta)
val_ds      = FrameDataset(FRAMES_DIR, val_meta)

train_loader= DataLoader(train_ds, batch_size=2, shuffle=True,  num_workers=2, pin_memory=True)  # changed batch size
val_loader  = DataLoader(val_ds,   batch_size=2, shuffle=False, num_workers=2, pin_memory=True)


Cell 8.1 — Imports & Device Setup

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm
import os

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Using device: {DEVICE}")


✅ Using device: cuda


Cell 8.2 — Define the Model

In [8]:
class ResNet50BiLSTM(nn.Module):
    def __init__(self, hidden=256):
        super().__init__()
        base = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
        self.cnn = nn.Sequential(*list(base.children())[:-1])
        self.lstm = nn.LSTM(2048, hidden, batch_first=True, bidirectional=True)
        self.head = nn.Sequential(
            nn.Linear(hidden * 2, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)
        feats = self.cnn(x).view(B, T, -1)
        lstm_out, _ = self.lstm(feats)
        last_out = lstm_out[:, -1, :]
        out = self.head(last_out)
        return out


Cell 8.3 — Initialize Model, Optimizer, Loss

In [9]:
model = ResNet50BiLSTM().to(DEVICE)
opt   = optim.Adam(model.parameters(), lr=1e-4)
crit  = nn.BCELoss()
epochs = 30  # changed from 50 → 30

model_save_dir = '/content/models'
os.makedirs(model_save_dir, exist_ok=True)

print("✅ Model, optimizer, and loss function initialized successfully.")


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 113MB/s]


✅ Model, optimizer, and loss function initialized successfully.


Cell 8.4 — Training & Validation Loop

In [10]:
for e in range(1, epochs + 1):
    model.train()
    train_loss = 0.0

    for vids, labels in tqdm(train_loader, desc=f"Epoch {e} Training"):
        vids, labels = vids.to(DEVICE), labels.to(DEVICE)
        labels = labels.view(-1, 1)

        preds = model(vids)
        loss = crit(preds, labels)

        opt.zero_grad()
        loss.backward()
        opt.step()

        train_loss += loss.item() * vids.size(0)

    train_loss /= len(train_loader.dataset)

    model.eval()
    val_loss, correct = 0.0, 0
    with torch.no_grad():
        for vids, labels in tqdm(val_loader, desc=f"Epoch {e} Validation"):
            vids, labels = vids.to(DEVICE), labels.to(DEVICE)
            labels = labels.view(-1, 1)

            preds = model(vids)
            l = crit(preds, labels)

            val_loss += l.item() * vids.size(0)
            correct += ((preds > 0.5).float() == labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_acc = correct / len(val_loader.dataset)

    print(f"Epoch {e}: Train Loss={train_loss:.4f} | Val Loss={val_loss:.4f} | Val Acc={val_acc:.4f}")

    torch.save(model.state_dict(), os.path.join(model_save_dir, f"model_epoch_{e}.pth"))


Epoch 1 Validation: 100%|██████████| 40/40 [00:07<00:00,  5.26it/s]


Epoch 1: Train Loss=0.5437 | Val Loss=0.4722 | Val Acc=0.8125


Epoch 2 Validation: 100%|██████████| 40/40 [00:06<00:00,  6.08it/s]


Epoch 2: Train Loss=0.4777 | Val Loss=0.4777 | Val Acc=0.8125


Epoch 3 Validation: 100%|██████████| 40/40 [00:07<00:00,  5.56it/s]


...

Epoch 28: Train Loss=0.0223 | Val Loss=1.0823 | Val Acc=0.7625


Epoch 29 Validation: 100%|██████████| 40/40 [00:06<00:00,  6.24it/s]


Epoch 29: Train Loss=0.0304 | Val Loss=1.2430 | Val Acc=0.7750


Epoch 30 Validation: 100%|██████████| 40/40 [00:07<00:00,  5.30it/s]


Epoch 30: Train Loss=0.0640 | Val Loss=1.1167 | Val Acc=0.7625
